# 실수

## 실수 연산의 함정

#### 실수 연산 예제

In [1]:
a = 0.01

In [2]:
result = 0.0

In [3]:
for i in range(100):
    result += a

In [8]:
result

1.0000000000000007

In [9]:
0.01*100

1.0

- 이 코드를 보면 a 가 0.01 이고 이를 100번 더했으니 당연히 1이 나오기를 기대함
    - 그러나 결과는 예상과 달리 나온다
- 1에 매우 가까운 수라고는 해도 1은 아니다

#### 다른 예제

In [10]:
a = 0.015625
a

0.015625

In [11]:
result = 0.0

In [12]:
for i in range(100):
    result += a

In [13]:
result

1.5625

In [14]:
0.015625 * 100

1.5625

- 이번에는 결과로 정확한 값이 나왔다
    - 이 결과를 이해하려면 컴퓨터가 실수를 표현하는 방법인 부동소수점이라는 개념이 이해해야 한다

## 부동소수점

- 컴퓨터는 IEEE(Institute of Electrical and Elecronics Engineers, 전기전가기술자협회)가 1985년에 제정한 ANSI/IEEE 754-1985라는 표준에 따라 실수(real number)를 표현한다
    - 이 표준에 따른 표현법을 부동소수점(floating-point)이라고 부른다
    - 부동소수점의 "부"는 부표를 말할 때 쓰는 "부"로 둥둥 떠다닌다는 의미

#### 실수 123.456을 여러 가지 방식으로 표현

- 1.23456 * $10^2$
- 12.3456 * $10^1$
- 1234.56 * $10^-1$
- 12345.6 * $10^-2$

- 소수점 위치를 보면 앞에 있기도 하고 뒤에 있기도 하다
    - 마치 소수점이 떠다니는 것처럼 보인다고 해서 이러한 실수 표현 방식을 부동소수점이라고 부른다

## 단정도와 배정도

- 부동소수점에는 단정도 부동소수점과 배정도 부동소수점이 있다
    - 단정도(single-precision)는 실수를 32비트(4바이트)로 표현하며 부호 1비트, 지수부 8비트, 가수부 23비트로 구성된다
    - 배정도(double-precision)는 실수를 64비트(8바이트)로 표현하며 부호 1비트, 지수부 11비트, 가수부 52비트로 구성된다
    - 배정도는 실수를 표현하는 데 사용하는 비트 수가 단정도보다 두 배 많은 만큼 정밀도가 높다
    - 파이썬은 배정도를 사용한다

#### 코드로 확인

In [15]:
import sys
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

- sys 모듈을 import한 다음 float_info를 보면 최댓값, 최솟값, 정밀도 등을 확인 할 수 있다

#### 배정도 부동소수점으로 표현할 수 있는 가장 큰 수

In [16]:
sys.float_info.max

1.7976931348623157e+308

- 여기서 e는 지수를 뜻하는 exponent의 줄임말
    - 배정도 부동소수점으로 표현할 수 있는 가장 큰 수를 다르게 표현 하면 1.7976931348623157*$10^{308}$

#### 배정도 부동소수점으로 표현할 수 있는 가장 작은 수

In [17]:
sys.float_info.min

2.2250738585072014e-308

- 10의 -308승이 얼마나 작은지 상상조차 할 수 없다
    - 그만큼 8바이트 배정도 부동소수점을 사용하면 표현 범위가 엄처나게 넓어진다

## 1바이트 실수 자료형 설계하기

- 실수 표현 방식을 쉽게 이해할 수 있도록 1바이트 실수 자료형을 직접 만들어 보자

#### 실수 자료형을 표현한 수식
- $± 1.man * 2^{exp-bias}$
    - 1.man 은 가수(mantissa/fraction), 2는 밑수, exp-bias는 지수(exponent)를 의미

### 1. 10진수 실수를 2진수 실수로 바꾸기

- 10진수 실수를 2진수로 바꾸는 방법은 정수를 바꿨던 방법과 크게 다르지 않다
- 여기서 $2^{-1}$은 음수가 아니라 우리가 흔히 분수라고 부르는 1/2이다, $2^{-2}$는 1/4이다

#### 예제
- 다음 수식처럼 쪼갠 다음 변환을 하면 111.11이 된다

- 7.75 = 4 + 2 + 1 + 0.5 + 0.25
- = $2^2 + 2^1 + 2^0 + 2^{-1} + 2^{-2}$
- = 111.11


### 2. 정규화

- 정규화(normalization)란 소수점 왼쪽에 위치한 가수 부분을 밑수보다 작은 자연수가 되도록 만드는 것
    - 예를 들어 10진수 567.89를 정규화하면 소수점 왼쪽에 위치한 가수 부분이 밑수 10보다 작은 자연수 5가 되어 5.6789 * $10^2$이 된다
    
- 2진수의 밑수는 2이므로 2보다 작은 자연수는 1밖에 없다
    - 따라서 소수점 왼쪽의 가수부분은 항상 1이 된다
    - 예를 들어 111.11을 정규화하면 111.11 = 1.111 * $2^2$

### 3. 메모리 구조
- 정규화된 부동소수점 수 1.1111 * $2^2$을 앞의 수식과 비교해 보면 man은 1111이고 exp-bias는 2이다
- 이제 1바이트의 메모리 구조를 정하고 man과 exp값만 저장하면 설계가 끝난다
- 이때 지수부와 가수부에 할당하는 비트 수에 따라 표현 범위와 정밀도가 결정된다

#### 1바이트 부동소수점 구성
- 0 0000 000
- 부호 지수부 가수부

- 실수 역시 정수와 마찬가지로 첫 번째 비트는 부호를 나타냄
    - 0이면 양수고 1이면 음수
    - 가운데 4비트는 지수부로 exp값 저장
    - 맨 뒤 3비트는 가수부로 man값 저장
    - bias 는 지수의 부호를 결정하는 데 쓰인다
- 부동소수점의 지수부에는 부호 비트가 없으며 0 ~ 15의 양수만 나타낼 수 있다
    - 하지만 음수 지수도 필요하다, 음수를 사용하려면 bias를 7로 두고 지수부(exp)에서 bias를 뺀 값을 실제 지수로 사용
        - bias는 $2^{n-1}$식에 지수부의 비트 수인 4를 대입하면 구할 수 있다

- 1.1111 * $2^2$에서 실제 지수는 2이다
- 실제 지수가 2라는 의미는 exp - bias가 2라는 것이다
- 지수부와 비트 수 4를 식($2^{n-1}$-1)에 대입해 얻은 bias 값이 7이므로 실제로 부동소수점의 지수부에 나타나는 값 exp는 9가 된다

#### 1바이트 구성

- 부호 -> 0
- 지수부 -> 1001
- 가수부 -> 1111

- 가수부는 3비트만 할당되는데 위에서 확인해보면 가수부 값이 1111이다
    - 이런 경우에는 뒷자리 1을 생략한다 즉, 가수부는 111이 된다
#### 0 1001 111 = 0100 1111 = 0x4f
- 실수 7.75는 1바이트 부동소수점으로 나타내면 0x4f이다

### 4. 1바이트 부동소수점의 표현 범위

- 지금까지 설계한 1바이트 부동소수점으로 표현할 수 있는 가장 작은 수와 가장 큰 수는 다음과 같다

- 표현할 수 있는 가장 작은 수(지수부가 0001일 때)
    - 1.000 * $2^{-6}$ = 0.0152625
- 표현할 수 있는 가장 큰 수(지수부가 1110일 떄)
    - 1.111 * $2^7$ = 240
- 1바이트지만 굉장히 작은 수부터 큰 수까지 폭넓게 표현할 수 있다

#### 단, 지수부 비트가 모두 0 일 때($2^{-7}$)와 모두 1일 때($2^8$)는 0.0, 정규화 불가능, 무한대, NaN(Not a Number, 숫자가 아님) 같은 특별한 상황을 나타내므로 제외

### 5. 1바이트 부동소수점의 정밀도
- 1바이트 부동소수점 설계를 마쳤는데 석연치 않은 점이 하나 있다
    - 변환 과정에서 가수부를 담을 공간이 부족해 가수부에 들어갈 데이터인 1111에서 맨 뒤에 있는 1을 누락한것
    - 이렇게 되면 0x4f는 7.75라는 실수를 완벽하게 표현하지 못한다
- 1.111 * $2^2$ = 1 * $2^2$ + 1 * $2^1$ + 1 * $2^0$ + 1 * $2^{-1}$ = 7.5
    - 0.25 만큼이나 차이가 나므로 정밀도가 그만큼 떨어진다

## 정밀도에 대한 고찰

### 1. 엡실론
- 실수 자료형에서 엡실론(epsilon)이란 1.0과 그 다음으로 표현 가능한 수(respresentable float)사이의 차이를 말합니다

#### 코드로 확인

In [19]:
sys.float_info.epsilon

2.220446049250313e-16

- 파이썬이 사용하는 배정도(double)의 가수부가 52비트
    - 1.0을 배정도에 맞춰 표현하면
    - 1.000......0000(0:52개) * $2^0$
- 배정도에서 1.0 다음으로 표현할 수 있는 수(representable float)은 다음과 같다
    - 1.000.....0001(0:51개, 1:마지막 비트) * $2^0$
- 따라서 두 수의 차이는 다음과 같다
    - 0.0000.....0001(0:51개, 1: 마지막 비트) * $2^0$
- 이 수를 10진수로 바꾸면 엡실론 값이 나온다

### 2. 엡실론과 정밀도
- 엡실론이 어떻게 쓰이는지
    - 어떤 실수가 있을 때 엡실론을 이용하면 그 실수 다음에 표현할 수 있는 수를 알아낼 수 있다
    - 예를 들어 배정도 실수 9.25를 부동소수점 방식으로 표현하면 1.00101 * $2^3$이다
        - 이 식에서 지수 부분만 뗴어 내 엡실론을 곱하면 이 실수와 다음 표현 가능한 수 사이의 차이를 구할 수 있다

#### 코드로 확인

In [26]:
ep = sys.float_info.epsilon

In [27]:
a = 9.25

In [28]:
diff = (2**3)*ep #1

In [29]:
diff

1.7763568394002505e-15

In [30]:
b = a +diff #2

In [31]:
b

9.250000000000002

- #1에서 diff는 지수 부분인 $2^3$에 엡실론을 곱한 값으로 9.25와 그 다음 표현 가능한 수 사이의 차이
- #2에서 b는 a에 diff를 더했으므로 9.25 다음에 표현 가능한 수를 나타낸다

#### 9.25에 diff보다 작은 값을 더하면 어떤 일이 일어날까?

In [33]:
a

9.25

In [34]:
half_diff = diff/2 #1

In [35]:
half_diff #2

8.881784197001252e-16

In [36]:
c = a + half_diff #3

In [37]:
a == c #4

True

- half_diff는 diff 값의 반절(#1).
- half_diff 역시 실수를 나눠서 얻은 값이니 실수(#2)
- a와 half_diff를 더해 c에 대입(#3)
- 실수와 실수를 더했으므로 c는 반드시 a보다 커야 한다
    - 그런데 a값과 c값이 같다고 나온다(#4)
        - a값과 c값이 같은 이유는 c에 더한 half_diff 값이 diff 보다 작기 때문
        - diff는 9.25와 그다음 표현 가능한 수 사이의 차이이므로 9.25에 diff보다 작은 값을 더한 수를 부동소수점 방식에서는 표현할 수 없다
        - 달리 표현하면 정밀도가 떨어진다는 말이다

#### 부동소수점의 정밀도에 관한 예 2

- a는 10000....0000(0:52개) * $2^{53}$이다
    - a와 a 다음에 표현 가능한 수 사이의 차이는 $2^{53}$에 엡실론을 곱해 구할 수 있으며 그 값은 2.0이다
        - 그러므로 a에 1.0을 더한 b는 원래 의도한 값을 표현하지 못하고 a와 같은 값을 갖게 됩니다
        - 1.0의 차이조차 표현할 수 없는 정밀도입니다.

#### 코드

In [38]:
a = (2.0)**53

In [39]:
a

9007199254740992.0

In [42]:
b = a + 1.0

In [44]:
a == b

True

## 마무리

- 부동소수점은 지수부에 따라 아주 작은 수와 아주 큰 수를 표현할 수 있지만, 어떤 상황에서는 16과 17 같은 큰 단위의 정수조차 제대로 표현할 수 없을 때도 있다
- "표현 범위는 넓지만 정밀도는 낮다"